In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [2]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [3]:
epochs.apply_baseline((None, 0))

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [4]:
reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV

In [5]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [6]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0,
                                                           average=True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1,
                                                           n_eeg=1, average=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15612\850005856.py:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


In [7]:
eog_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203, exp. var : 92.61%>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102, exp. var : 88.38%>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59, exp. var : 98.97%>]

In [8]:
projs = eog_projs + ecg_projs
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203, exp. var : 92.61%>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102, exp. var : 88.38%>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59, exp. var : 98.97%>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203, exp. var : 78.55%>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102, exp. var : 94.80%>]

In [9]:
epochs.add_proj(projs)


Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [ ]:
epochs_cleaned = epochs.copy().apply_proj()

#epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks='EEG 060')

In [21]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40)  # High-pass with 1. Hz cut-off is recommended for ICA

C:\Users\Dell\AppData\Local\Temp\ipykernel_15612\699562409.py:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


<Raw | sub-01_ses-01_task-audiovisual_run-01_meg.fif, 376 x 166800 (277.7 s), ~478.9 MB, data loaded>

In [22]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [23]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

In [ ]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True)

In [ ]:
epochs_ica.info

In [ ]:
n_components = 0.8  # Should normally be higher, like 0.999!!
method = 'picard'
max_iter = 100  # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

ica = mne.preprocessing.ICA(n_components=n_components,
    max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
ica.fit(epochs_ica)


In [ ]:
ica.plot_components(inst=epochs)

In [18]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
    ecg_epochs, method='ctps')


eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
    eog_epochs)

components_to_exclude = ecg_inds + eog_inds
ica.exclude = components_to_exclude

NameError: name 'ica' is not defined

In [19]:
ica.plot_scores(ecg_scores)

NameError: name 'ica' is not defined

In [ ]:
ica.plot_sources(ecg_evoked)

In [20]:
ica.plot_overlay(ecg_evoked)

NameError: name 'ica' is not defined